# 2487-2223 Machine Learning Assignment 1



## Question 1 (25 points) - Zestimate this House

Purchasing a house is a very big decision for most of us. Companies such as Zillows collected tons of data regarding the listing and sold price of American houses and build the predictive model, named *Zestimate*. You are expected to build a model similar as Zestimate to predict house price in Boston. 

![zestimate](https://i0.wp.com/www.housesoldeasy.com/wp-content/uploads/Screen-Shot-2016-08-15-at-7.22.09-PM.png?resize=300%2C258&ssl=1)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LogisticRegressionCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error

In [2]:
### DON'T MODIFY - LOAD DATA ### 

data_url = "http://lib.stat.cmu.edu/datasets/boston" 
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]]) # FEATURES 
y = raw_df.values[1::2, 2] # TARGET VARIABLE
assert X.shape[0] == y.shape[0], 'Mismatch in number of examples.'
print('Data loaded correctly.')
print('Features: X. Target variable (price): y.')
print('X shape: ',X.shape, 'y shape: ', y.shape)
### END ###

Data loaded correctly.
Features: X. Target variable (price): y.
X shape:  (506, 13) y shape:  (506,)


The columns:
| Index | Variable | Description                                                 |
|-------|----------|-------------------------------------------------------------|
| 0     | CRIM     | per capita crime rate by town                              |
| 1     | ZN       | proportion of residential land zoned for lots over 25,000 sq.ft. |
| 2     | INDUS    | proportion of non-retail business acres per town             |
| 3     | CHAS     | Charles River dummy variable (= 1 if tract bounds river; 0 otherwise) |
| 4     | NOX      | nitric oxides concentration (parts per 10 million)           |
| 5     | RM       | average number of rooms per dwelling                         |
| 6     | AGE      | proportion of owner-occupied units built prior to 1940       |
| 7     | DIS      | weighted distances to five Boston employment centres        |
| 8     | RAD      | index of accessibility to radial highways                    |
| 9     | TAX      | full-value property-tax rate per $10,000                     |
| 10    | PTRATIO  | pupil-teacher ratio by town                                  |
| 11    | B        | 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town |
| 12    | LSTAT    | % lower status of the population                             |
| 13    | MEDV     | Median value of owner-occupied homes in $1000's              |

Label = y = MEDV (price)


#### Question 1.1 (5 points) 
Create train and test set, each contains 80% and 20% of the dataset, respectively, using *train_test_split* function in scikit-learn. Train a linear model on the train set and evaluate on the test set, report the training error and test error, respectively (as mean squared error).

In [3]:
seed = 13

# create linear regression object
reg = LinearRegression()

# split data into training and testing data sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = seed)

# train the model with training set to find the best parameters for the best-fitting line
reg.fit(X_train, y_train)

# make predictions with train set and compute MSE on train data
y_pred_test = reg.predict(X_train)
error_train = mean_squared_error(y_train, y_pred_test)
    
# make predictions with test data and compute MSE on test data (unseen)
y_pred = reg.predict(X_test)
error_test = mean_squared_error(y_test, y_pred)

print('Training error:', error_train) # how well the model fits the training data
print('Test error:', error_test) # how well the model can predict new, unseen data

Training error: 21.555648194527308
Test error: 24.318238309170436


#### Question 1.2 (5 points)

Perform a 10-fold cross-validation on the whole data set. Show the averaged mean sqaured error on both train and test set at each fold. Explain your findings.

In [4]:
# perform 10-fold cross-validation on dataset with MSE
scores = cross_validate(reg, X, y, scoring='neg_mean_squared_error', cv=10, return_train_score = True)

# get train and test MSE from scores dict, multiply by -1 to get positive MSE
train_MSE = scores["train_score"] * -1  # The score array for train scores on each cv split
test_MSE = scores["test_score"] * -1    # The score array for test scores on each cv split

# create dataframe containing the average MSE for train and test set for each fold
mse_df = pd.DataFrame({'train_MSE': train_MSE, 'test_MSE': test_MSE})
mse_df

,train_MSE,test_MSE
0,23.363228,9.286947
1,22.882034,14.151283
2,23.216075,14.073606
3,20.771703,35.206924
4,21.335426,31.885117
5,22.363700,19.835878
6,23.327221,9.947269
7,11.959197,168.375380
8,21.586295,33.329745
9,23.189043,10.960411


In [5]:
print("Average training error over 10-fold cross-validation:", train_MSE.mean())
print("Average testing error over 10-fold cross-validation:", test_MSE.mean())

Average training error over 10-fold cross-validation: 21.39939241710581
Average testing error over 10-fold cross-validation: 34.70525594452485


Findings: the model's performance is not consistent across folds. The test error is in most folds smaller than the training error, which is an indicator that in those folds the model does generalize well to new data. 
<span style="color:red; font-size:20px">Do it later</span>


#### Question 1.3 (5 points) 
 
Add 2-degree squared polynomial features (with no interactions) and perform 10-fold cross-validation on the whole data set. Show the mean sqaured error on both train and test set at each fold. Explain your findings.

Hint: you may use sklearn.preprocessing.PolynomialFeatures and check how it produces the polynomial features with/without interaction terms.

#####  Trying out behavior of interaction terms

In [6]:
X.shape

(506, 13)

In [7]:
poly_interaction = PolynomialFeatures(degree = 2, interaction_only = True)
X_interaction = poly_interaction.fit_transform(X)
print(X_interaction.shape)
print("Only interaction features (the product of different input features) and the terms with power 1 are produced")

(506, 92)
Only interaction features (the product of different input features) and the terms with power 1 are produced


In [8]:
poly_no_interaction = PolynomialFeatures(degree = 2, interaction_only = False)
X_no_interaction = poly_no_interaction.fit_transform(X)
print(X_no_interaction.shape)
print("Includes 78 interaction features, 13 features with power of 1, 13 features with power of 2, and intercept term")

(506, 105)
Includes 78 interaction features, 13 features with power of 1, 13 features with power of 2, and intercept term


In [9]:
new_X = np.hstack((X, X**2))
new_X.shape

(506, 26)

Note: to achieve no interaction, one must transform X by using np.hstack. I understood it in a way that you want no interaction features at all.

In [10]:
# save an instance of PolynomialFeatures with the given degree
polynomial_features = PolynomialFeatures(degree = 2, interaction_only = False)

# create polynomial regression model
model = LinearRegression()

# fit model to data, using the 
model.fit(new_X, y)

# perform 10-fold cross-validation on dataset with MSE
scores_poly = cross_validate(model, new_X, y, scoring='neg_mean_squared_error', cv=10, return_train_score = True)

# get train and test MSE from scores dict, multiply by -1 to get positive MSE
train_MSE_poly = scores_poly["train_score"] * -1 
test_MSE_poly = scores_poly["test_score"] * -1 

# create dataframe containing the average MSE for train and test set for each fold
mse_df_poly = pd.DataFrame({'train_MSE': train_MSE_poly, 'test_MSE': test_MSE_poly})
mse_df_poly


,train_MSE,test_MSE
0,14.980271,10.091861
1,14.969661,8.791095
2,15.204169,11.458221
3,13.665059,22.518524
4,14.438738,14.223065
5,14.940824,8.780146
6,14.859624,12.953433
7,7.419826,104.037379
8,14.605032,13.610872
9,14.050919,50.771795


In [11]:
print("Average training error over 10-fold cross-validation:", train_MSE_poly.mean())
print("Average testing error over 10-fold cross-validation:", test_MSE_poly.mean())

Average training error over 10-fold cross-validation: 13.913412317032257
Average testing error over 10-fold cross-validation: 25.72363917701756


<span style="color:red; font-size:20px">Explain your findings.</span>

#### Question 1.4 (10 points)

Perform cross-validation using ridge regression and lasso regression on different feature combinations (linear features vs. polynomial features obtained earlier respectively. Explain which method works better in this case. Check the coefficients and explain the differences between ridge regression and lasso regression.

##### Linear features

In [21]:
# Ridge Regression
X_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']

ridge_reg = Ridge()
cv_scores_rl = cross_validate(ridge_reg, X, y, cv=10, scoring='neg_mean_squared_error', return_train_score=True)

# get train and test MSE from scores dict, multiply by -1 to get positive MSE
train_MSE_rl = cv_scores_rl["train_score"] * -1 
test_MSE_rl = cv_scores_rl["test_score"] * -1 

print("Ridge Regression - Linear Features:")
print("Average training error over 10-k cv with ridge regression (linear):", train_MSE_rl.mean())
print("Average testing error over 10-k cv with ridge regression (linear):", test_MSE_rl.mean())

# Create dataframe with feature names and coefficients
ridge_reg.fit(X_train, y_train)
ridge_coefs = pd.DataFrame({'feature': X_names, 'coefficient': ridge_reg.coef_})
print("Coefficients")
print(ridge_coefs)

Ridge Regression - Linear Features:
Average training error over 10-k cv with ridge regression (linear): 21.58114506497026
Average testing error over 10-k cv with ridge regression (linear): 34.07824620925931
Coefficients
    feature  coefficient
0      CRIM    -0.150575
1        ZN     0.058798
2     INDUS    -0.035243
3      CHAS     2.424039
4       NOX    -8.099813
5        RM     3.543438
6       AGE    -0.002831
7       DIS    -1.472918
8       RAD     0.351650
9       TAX    -0.015446
10  PTRATIO    -0.821765
11        B     0.007758
12    LSTAT    -0.570019


In [26]:
# Lasso regression
lasso_reg = Lasso(max_iter=5000)

cv_scores_ll = cross_validate(lasso_reg, X, y, cv=10, scoring='neg_mean_squared_error', return_train_score=True)

# get train and test MSE from scores dict, multiply by -1 to get positive MSE
train_MSE_ll = cv_scores_ll["train_score"] * -1 
test_MSE_ll = cv_scores_ll["test_score"] * -1 

print("\nLasso Regression - Linear Features:")
print("Average training error over 10-k cv with lasso regression (linear):", train_MSE_ll.mean())
print("Average testing error over 10-k cv with lasso regression (linear):", test_MSE_ll.mean())

lasso_reg.fit(X_train, y_train)
lasso_coefs = pd.DataFrame({'feature': X_names, 'coefficient': lasso_reg.coef_})
print("Coefficients")
print(lasso_coefs)


Lasso Regression - Linear Features:
Average training error over 10-k cv with lasso regression (linear): 26.446016110077032
Average testing error over 10-k cv with lasso regression (linear): 34.46408458830231
Coefficients
    feature  coefficient
0      CRIM    -0.100439
1        ZN     0.061775
2     INDUS    -0.000000
3      CHAS     0.000000
4       NOX    -0.000000
5        RM     0.737827
6       AGE     0.025951
7       DIS    -0.752248
8       RAD     0.334002
9       TAX    -0.018118
10  PTRATIO    -0.694069
11        B     0.006781
12    LSTAT    -0.793181


##### Polynomial features

In [24]:
# Ridge Regression
cv_scores_rp = cross_validate(ridge_reg, new_X, y, cv=10, scoring='neg_mean_squared_error', return_train_score=True)

# get train and test MSE from scores dict, multiply by -1 to get positive MSE
train_MSE_rp = cv_scores_rp["train_score"] * -1 
test_MSE_rp = cv_scores_rp["test_score"] * -1 

print("Ridge Regression - Polynomial Features:")
print("Average training error over 10-k cv with ridge regression (poly):", train_MSE_rp.mean())
print("Average testing error over 10-k cv with ridge regression (poly):", test_MSE_rp.mean())

Ridge Regression - Polynomial Features:
Average training error over 10-k cv with ridge regression (poly): 14.201739336116793
Average testing error over 10-k cv with ridge regression (poly): 26.942921093602394


In [27]:
# Lasso regression
cv_scores_lp = cross_validate(lasso_reg, new_X, y, cv=10, scoring='neg_mean_squared_error', return_train_score=True)

# get train and test MSE from scores dict, multiply by -1 to get positive MSE
train_MSE_lp = cv_scores_lp["train_score"] * -1 
test_MSE_lp = cv_scores_lp["test_score"] * -1 

print("\nLasso Regression - Polynomial Features:")
print("Average training error over 10-k cv with lasso regression (poly):", train_MSE_lp.mean())
print("Average testing error over 10-k cv with lasso regression (poly):", test_MSE_lp.mean())



Lasso Regression - Polynomial Features:
Average training error over 10-k cv with lasso regression (poly): 18.548501008872663
Average testing error over 10-k cv with lasso regression (poly): 30.776932337350466


In [13]:
# # Ridge Regression

# # use numpy array to figure out the best alpha
# alphas = np.linspace(0.0001,15,100) # array with 100 values, evenly spaced from 0.0001 to 15
# X_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']

# # Ridge regression - linear features
# ridge_linear = RidgeCV(alphas = alphas, cv = 10, scoring='neg_mean_squared_error').fit(X_train, y_train)
# print("Ridge Regression - Linear Features:")
# print("Best alpha:", ridge_linear.alpha_)
# print("R^2 Scores (best possible score: 1.0):")
# print("  Training R^2 score:", ridge_linear.score(X_train, y_train))
# print("  Testing R^2 score:", ridge_linear.score(X_test, y_test))

# # Create dataframe with feature names and coefficients
# ridge_coefs = pd.DataFrame({'feature': X_names, 'coefficient': ridge_linear.coef_})
# print("Coefficients")
# print(ridge_coefs)

In [64]:


# Lasso regression - linear features
lasso_linear = LassoCV(alphas = alphas, cv=10).fit(X_train, y_train)
print("\nLasso Regression - Linear Features:")
print("Best alpha:", lasso_linear.alpha_)
print("R^2 Scores (best possible score: 1.0):")
print("  Training score:", lasso_linear.score(X_train, y_train))
print("  Testing score:", lasso_linear.score(X_test, y_test))
lasso_coefs = pd.DataFrame({'feature': X_names, 'coefficient': ridge_linear.coef_})
print("Coefficients")
print(lasso_coefs)

Ridge Regression - Linear Features:
Best alpha: 0.0001
R^2 Scores (best possible score: 1.0):
  Training R^2 score: 0.7434997531265819
  Testing R^2 score: 0.7112250981524468
Coefficients
    feature  coefficient
0      CRIM    -0.133469
1        ZN     0.035809
2     INDUS     0.049516
3      CHAS     3.119810
4       NOX   -15.415576
5        RM     4.057204
6       AGE    -0.010822
7       DIS    -1.385976
8       RAD     0.242724
9       TAX    -0.008702
10  PTRATIO    -0.910669
11        B     0.011794
12    LSTAT    -0.547116

Lasso Regression - Linear Features:
Best alpha: 0.0001
R^2 Scores (best possible score: 1.0):
  Training score: 0.7434997098661364
  Testing score: 0.7112078878848566
Coefficients
    feature  coefficient
0      CRIM    -0.133469
1        ZN     0.035809
2     INDUS     0.049516
3      CHAS     3.119810
4       NOX   -15.415576
5        RM     4.057204
6       AGE    -0.010822
7       DIS    -1.385976
8       RAD     0.242724
9       TAX    -0.008702
10  PT

### Polynomial features

In [54]:
# Ridge regression on polynomial features
X_poly_train, X_poly_test, y_poly_train, y_poly_test = train_test_split(X_poly, y, test_size = 0.3, random_state = seed)
ridge_poly = RidgeCV(alphas = alphas, cv = 5)
ridge_poly.fit(X_poly_train, y_poly_train)
print("\nRidge Regression - Polynomial Features:")
print("Best alpha:", ridge_poly.alpha_)
print("R^2 Scores (best possible score: 1.0):")
print("  Training score:", ridge_poly.score(X_poly_train, y_poly_train))
print("  Testing score:", ridge_poly.score(X_poly_test, y_poly_test))
print("Coefficients:", ridge_poly.coef_)

/Users/hannah/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=8.4887e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/Users/hannah/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=7.95853e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/Users/hannah/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=7.97673e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/Users/hannah/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=7.64914e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/Users/hannah/opt/anaconda3/lib/p


Ridge Regression - Polynomial Features:
Best alpha: 0.1516141414141414
R^2 Scores (best possible score: 1.0):
  Training score: 0.936796553905228
  Testing score: 0.8135697365425578
Coefficients: [ 0.00000000e+00  4.70368446e-01 -4.84648561e-01 -4.57270727e+00
  2.56287060e+00 -1.99393260e-01  8.09536353e+00  1.10625670e+00
  5.84728151e-02  2.53638412e+00 -4.42229868e-02 -6.53926651e+00
 -3.39219173e-02  1.16446967e+00  3.64139952e-03  1.49636634e-01
  4.51835122e-01  2.10784726e+00 -5.08291577e-01  7.61673747e-02
 -6.91226220e-03 -1.29327463e-01  3.63342130e-01 -3.35257892e-02
  2.43145388e-01 -7.68608162e-04  2.88914313e-02 -8.65354234e-04
 -4.93961476e-03 -6.59892552e-02 -1.08696716e+00  2.16215359e-02
 -1.25730491e-04 -2.66974597e-03 -1.37977588e-03  7.17271290e-04
 -1.27738255e-02  2.48910417e-03 -4.90848121e-03  4.73572481e-02
 -5.66203012e-02 -1.61083727e+00  3.20201055e-01  2.33849585e-03
  8.46979628e-02  2.04981479e-02 -6.67282184e-04 -5.38601777e-02
  7.61901723e-03  1.019

In [56]:
# Lasso regression on polynomial features
lasso_poly = LassoCV(alphas = alphas, cv=5)
lasso_poly.fit(X_poly_train, y_poly_train)
print("\nLasso Regression - Polynomial Features:")
print("Best alpha:", lasso_poly.alpha_)
print("R^2 Scores (best possible score: 1.0):")
print("  Training score:", lasso_poly.score(X_poly_train, y_poly_train))
print("  Testing score:", lasso_poly.score(X_poly_test, y_poly_test))
print("Coefficients:", lasso_poly.coef_)

/Users/hannah/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1749.9074608942153, tolerance: 2.580162240282686
  model = cd_fast.enet_coordinate_descent_gram(
/Users/hannah/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 172.85783240298133, tolerance: 2.580162240282686
  model = cd_fast.enet_coordinate_descent_gram(
/Users/hannah/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 25.851389556127288, tolerance: 2.580162240282686
  model = cd_fast.enet_coordinate_descent_gram(
/Users/hannah/opt/anaconda3/lib/python3.9/site-packa


Lasso Regression - Polynomial Features:
Best alpha: 15.0
R^2 Scores (best possible score: 1.0):
  Training score: 0.8598141218210978
  Testing score: 0.797639859147076
Coefficients: [ 0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00  2.11759395e-04  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.19326560e-04
 -0.00000000e+00  3.06978935e-04  0.00000000e+00  8.76631326e-04
 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00  3.09174146e-04
  0.00000000e+00 -3.34561713e-04 -0.00000000e+00  0.00000000e+00
  0.00000000e+00 -0.00000000e+00 -0.00000000e+00  1.97775467e-04
 -0.00000000e+00  0.00000000e+00  3.92408374e-04 -0.00000000e+00
 -5.79025172e-04 -0.00000000e+00  0.0

/Users/hannah/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1292.554057219887, tolerance: 2.4273837323943668
  model = cd_fast.enet_coordinate_descent_gram(
/Users/hannah/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1400.880180486372, tolerance: 2.4273837323943668
  model = cd_fast.enet_coordinate_descent_gram(
/Users/hannah/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1381.717382901359, tolerance: 2.4273837323943668
  model = cd_fast.enet_coordinate_descent_gram(
/Users/hannah/opt/anaconda3/lib/python3.9/site-packa

## Question 2 (25 points) - Cancer Detection

Given a dataset with features that are computed from a digitized image of a fine needle aspirate (FNA) of a breast mass, which describes characteristics of the cell nuclei present in the image, let's try to predict whether the patients are diagnosed as Malignant (M) or Benign (B).

In [60]:
# LOAD DATA 

from sklearn.datasets import load_breast_cancer
"""
DOCS:
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html#sklearn.datasets.load_breast_cancer
"""
X, y = load_breast_cancer(return_X_y=True)

#### Question 2.1 (5 points) 
Use logistic regression to train the dataset through cross-validation, report the score on train and test set, respectively. Explain your findings.

In [78]:
# instantiate the model (using the default parameters)
logreg = LogisticRegression(max_iter=10000)

# fit the model with data
# logreg.fit(X, y)
# y_pred=logreg.predict(X_test)

# perform 10-fold cross-validation on dataset with MSE
scores_log = cross_validate(logreg, X, y, scoring='recall', cv=10, return_train_score = True)

# get train and test recall from scores dict
train_MSE_log = scores_log["train_score"]
test_MSE_log = scores_log["test_score"]

# create dataframe containing the average MSE for train and test set for each fold
# mse_df_log = pd.DataFrame({'train_MSE': train_MSE_log, 'test_MSE': test_MSE_log})
# mse_df_log

print("Mean Recall on Training set:", train_MSE_log.mean())
print("Mean Recall on Testing set:", test_MSE_log.mean())

Mean Recall on Training set: 0.9754116599910991
Mean Recall on Testing set: 0.9721428571428572


#### Question 2.2 (5 points) 
By default, sklearn's logistic regression uses the L2 regularization. Now use the logistic regression without any regularzation to perform cross validation, report what do you find on train and test set.

In [85]:
# instantiate the model
logreg2 = LogisticRegression(max_iter=10000, penalty='none')

# fit the model with data
logreg2.fit(X, y)
# y_pred=logreg.predict(X_test)

# perform 10-fold cross-validation on dataset with MSE
#scores_log = cross_validate(logreg2, X, y, scoring='recall', cv=5, return_train_score = True)

# get train and test recall from scores dict
# train_MSE_log = scores_log["train_score"]
# test_MSE_log = scores_log["test_score"]

# Train and evaluate model using 5-fold cross-validation
scores_train = cross_val_score(logreg2, X, y, cv=5)
#scores_test = cross_val_score(logreg2, X_test, y_test, cv=5)

# create dataframe containing the average MSE for train and test set for each fold
# mse_df_log = pd.DataFrame({'train_MSE': train_MSE_log, 'test_MSE': test_MSE_log})
# mse_df_log

#print("Mean Recall on Training set:", train_MSE_log.mean())
#print("Mean Recall on Testing set:", test_MSE_log.mean())

/Users/hannah/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/hannah/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#log

Mean Recall on Training set: 0.9922954238743712
Mean Recall on Testing set: 0.9746870109546165


#### Question 2.3 (15 points) 
Check how many Benign and Malignant cases in the dataset. What might be the problem if we use the default score of the logistic regression cross-validation? Now adjust the class weight of M and L and retrain the model again to bias toward Malignant, using the relative weight of M and L as 2:1. What about the relaive weight to be 5:1, or 10:1? Explain what you find.

Hint: you can use LogisticRegressionCV to combine LogisticRegression and cross-validation. 

In [90]:
unique, counts = np.unique(y, return_counts=True)
dict(zip(unique, counts))

{0: 212, 1: 357}

0 = negative class = Malignant (=Cancer)

1 = positive class = Benign

see class distribution as stated here: https://scikit-learn.org/stable/datasets/toy_dataset.html#breast-cancer-dataset

In [92]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Count number of samples in each class
n_samples = y.shape[0]
n_malignant = sum(y == 0)
n_benign = sum(y == 1)

# 0 = negative class = Malignant (=Cancer)
# 1 = positive class = Benign

print(f"Number of malignant cases: {n_malignant} ({n_malignant / n_samples:.2%})")
print(f"Number of benign cases: {n_benign} ({n_benign / n_samples:.2%})")

# Define logistic regression model with class weights
clf_2to1 = LogisticRegressionCV(class_weight={0: 1, 1: 2}, cv=5, max_iter=10000)
clf_5to1 = LogisticRegressionCV(class_weight={0: 1, 1: 5}, cv=5, max_iter=10000)
clf_10to1 = LogisticRegressionCV(class_weight={0: 1, 1: 10}, cv=5, max_iter=10000)

# Train model with 2:1 class weight ratio
clf_2to1.fit(X_train, y_train)

# Evaluate model on test set
score_2to1 = clf_2to1.score(X_test, y_test)
print(f"Score with 2:1 class weight ratio: {score_2to1:.2f}")

# Train model with 5:1 class weight ratio
clf_5to1.fit(X_train, y_train)

# Evaluate model on test set
score_5to1 = clf_5to1.score(X_test, y_test)
print(f"Score with 5:1 class weight ratio: {score_5to1:.2f}")

# Train model with 10:1 class weight ratio
clf_10to1.fit(X_train, y_train)

# Evaluate model on test set
score_10to1 = clf_10to1.score(X_test, y_test)
print(f"Score with 10:1 class weight ratio: {score_10to1:.2f}")

Number of malignant cases: 212 (37.26%)
Number of benign cases: 357 (62.74%)
Score with 2:1 class weight ratio: 0.99
Score with 5:1 class weight ratio: 0.96


/Users/hannah/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Score with 10:1 class weight ratio: 0.97


## Question 3 (50 points) - Call Me Maybe? 



![telemarketing](https://neilpatel.com/wp-content/uploads/2019/08/profissional-de-telemarketing-sorridente.jpeg)

Telemarketing is a method of direct marketing in which a salesperson solicits prospective customers to buy products or services over the phone. It has become one of the most widely used marketing campaign methods to engage with customers with product and service opportunity. We have collected real data from a Portuguese retail bank, from May 2008 to June 2013 with thousands of phone contacts. 




The current practice of many data teams is to build such propensity models and predict customer's probability to adopt the product and target them from the highest probability to the lowest probability. Note that telemarketing may incur some costs for contacting the customer, thus the success (i.e., the generated profit) of using machine learning model requries further inspection.  As the data scientist, you are asked to build a propensity model to evaluate the effectiveness of their telemarketing campaigns, i.e. whether the customer subscribed to the term deposit.  

**Telemarketing Dataset (bank.csv)**
All customers are contained in the file bank.csv. Each line of this file after the header row represents one customer of the Portuguese bank, and has the following format:

### bank client data:
- age (numeric)
- job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
- marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
- education (categorical: 'primary', 'secondary', 'tertiary')
- balance: amcount of bank account balance
- default: has credit in default? (categorical: 'no','yes','unknown')
- housing: has housing loan? (categorical: 'no','yes','unknown')
- loan: has personal loan? (categorical: 'no','yes','unknown')

### related with the last contact of the current campaign:
- contact: contact communication type (categorical: 'cellular','telephone', 'unknown')
- day: last contact day of month
- month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
- duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. 

### other attributes:
- campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
- pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; -1 means client was not previously contacted)
- previous: number of contacts performed before this campaign and for this client (numeric)
- poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
- y - has the client subscribed a term deposit? (binary: 'yes','no')


Answer the following questions using the provided dataset. You can write down intermediate results towards the final answers. If any model invovles random_state, set it to be 42.

In [3]:
bank = pd.read_csv('bank.csv', sep=';')
bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [4]:
bank.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

In [5]:
bank.describe()

,age,balance,day,duration,campaign,pdays,previous
count,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000
mean,41.170095,1422.657819,15.915284,263.961292,2.793630,39.766645,0.542579
std,10.576211,3009.638142,8.247667,259.856633,3.109807,100.121124,1.693562
min,19.000000,-3313.000000,1.000000,4.000000,1.000000,-1.000000,0.000000
25%,33.000000,69.000000,9.000000,104.000000,1.000000,-1.000000,0.000000
50%,39.000000,444.000000,16.000000,185.000000,2.000000,-1.000000,0.000000
75%,49.000000,1480.000000,21.000000,329.000000,3.000000,-1.000000,0.000000
max,87.000000,71188.000000,31.000000,3025.000000,50.000000,871.000000,25.000000


In [6]:
bank.y.value_counts()

no     4000
yes     521
Name: y, dtype: int64

In [97]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np

### Question 3.1 (15 points)

Split the data into 80% training set and 20% test set. **Build a pipeline to preprocess the indicated numerical features and categorical features separately**. For numerical features 'balance', 'campaign', standardize these features. For categorical features 'job', 'marital', 'education', 'default', transform them through one-hot encoding. For the numeric feature 'age', convert it into the quartile categorical variable and transform it through one-hot encoding. 

Train a Logistic regression model with L2 regularization using 5-fold cross validation (default hyperparameter) on the train set and show the accuracy, precision, recall on the train set. Explain whether the model is useful for the bank to identify the customer propensity.

In [57]:
X = bank.drop('y', axis = 1)
y = bank["y"] .replace({"yes": 1, "no": 0})

In [58]:
from sklearn import set_config
set_config(display="diagram")

In [62]:
seed = 42

# define attributes
num_attribs = ["balance", "campaign"]
cat_attribs = ["job", "marital", "education", "default"]

# define pipeline to apply feature scaling to numerical features
num_pipeline = Pipeline([
        ('std_scaler', StandardScaler())
])

# define pipeline to apply one-hot encoding to categorical values
cat_pipeline = Pipeline([
        ('cat_encoder', OneHotEncoder(sparse=False, handle_unknown='ignore')),
    ])

# define pipeline to convert age into the quartile categorical variable and transform it through one-hot encoding. 
age_pipeline = Pipeline([
        ('quartile_transformer', QuantileTransformer(n_quantiles=4)),
        ('cat_encoder', OneHotEncoder(sparse=False, handle_unknown='ignore')),
    ])

# apply different operations on num and cat by combining pipelines with ColumnTransformer
preprocessing = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
        ("age", age_pipeline, ["age"])
    ])

# append logistic regression to preprocessing pipeline
full_pipeline_l2 = Pipeline(steps=[
    ('preprocessor', preprocessing),
    ('classifier', LogisticRegression(penalty='l2'))
])

# split data into training and testing data sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=seed)

# perform cross-validation on the training data
scoring = ['accuracy', 'precision', 'recall']
cv_scores = cross_validate(full_pipeline_l2, X_train, y_train, cv=5, scoring=scoring)

# display mean accuracy, precision, and recall over the 5 folds
print("Accuracy:", cv_scores['test_accuracy'].mean())
print("Precision:", cv_scores['test_precision'].mean())
print("Recall:", cv_scores['test_recall'].mean())

Accuracy: 0.8832962716734294
Precision: 0.6
Recall: 0.009439775910364146


EXPLAIN...!!

### Question 3.2 (10 points)

Now add more features to the model to see if we can improve the performance (categorical features: 'housing', 'loan' and numerical features: 'day', 'duration'). Use the preprocess pipeline built previously to transform the data. 

Train a Logistic regression model with L1 regularization using 5-fold cross validation on the train set, by fine-tuning the hyperparameter alpha, i.e. the regularization strength from [0.001, 0.01, 0.1, 1]. Choose the correct score function that reflect the current data team's practice. Report the average score with the best hyperparameter. Does model performance improve, and if so, how?

Expalin whether all features are useful for making prediction and why. List top 5 features that contribute to the prediction the most. If not all features are useful, list those unuseful features.

In [70]:
# add more attributes
num_attribs = ["balance", "campaign", "day", "duration"]
cat_attribs = ["job", "marital", "education", "default", "housing", "loan"]

# append logistic regression to preprocessing pipeline
full_pipeline_l1 = Pipeline(steps=[
    ('preprocessor', preprocessing),
    ('classifier', LogisticRegression(penalty='l1'))
])

# split data into training and testing data sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=seed)

# Use the GridSearchCV object with cross_val_score to perform cross-validation and hyperparameter tuning
params = {'C': [0.001, 0.01, 0.1, 1]}
grid_search = GridSearchCV(full_pipeline_l1, params)

#grid_search.best_params_
# scoring = ['accuracy', 'precision', 'recall']
# cv_scores2 = cross_validate(grid_search, X_train, y_train, cv=5, scoring=scoring)

# cv_scores2

### Question 3.3 (10 points)

Now use the best model found in the cross-validation to predict the test set, show the obtained confusion matrix. Assume that targeting each customer would cost 10 euros and if the customer subscribe, the company would earn 50 euros. If we perform targeted telemarketing to all customers that are predicted to subscribe in the test set, what's the resulting profit?

### Question 3.4 (10 points)

Now adjust the decision threshold in order to optimize the obtained profit. What would be the resulting threshold and profit? Is the propensity model built based on the targeting predicted probability useful in terms of profit maximizing? Explain.

### Question 3.5 (5 points)

Now train a random forest model, with 10 decision trees and max_depth=2, what is the profit that can be achieved given the threshold that you identified earlier? Do you need to increase or decrese the threshold to maximize the profit using random forest model? Explain your result.